In [1]:
import torch

from seq2seq.data import Dataset
from seq2seq.model import Seq2seqModel
from seq2seq.train import Trainer, Evaluator

In [2]:
GPU_ID = 1
device = torch.device("cuda:{}".format(GPU_ID)) if torch.cuda.is_available() else torch.device("cpu")

# 1. Preprocess Data

In [3]:
SRC_FILE_PATH = 'smart_mobile/smart_src_khaiii.txt'
TGT_FILE_PATH = 'smart_mobile/smart_tgt_khaiii.txt'

EXPR_PATH = 'smart_mobile/'

BATCH_SIZE = 32
MAX_SRC_LEN = 10
MAX_TGT_LEN = 10

In [4]:
dataset = Dataset(
    src_file_path=SRC_FILE_PATH,
    tgt_file_path=TGT_FILE_PATH,
    max_src_len=MAX_SRC_LEN,
    max_tgt_len=MAX_TGT_LEN
)

Reading lines...
Read 90729 sentence pairs

Trim data to 20074 sentence pairs
Avg length of src :  7.272591411776427
Avg length of tgt :  7.307063863704294

Source vocab : 5062 (0 reduced)
Target vocab : 5060 (0 reduced)

Success to preprocess data!



# 2. Define Model

In [5]:
NUM_LAYERS = 1
INPUT_SIZE = dataset.src_vocab_size
EMBED_SIZE = 64
HIDDEN_SIZE = 64
OUTPUT_SIZE = dataset.tgt_vocab_size

In [6]:
model = Seq2seqModel(
    n_layers=NUM_LAYERS,
    input_size=INPUT_SIZE,
    emb_size=EMBED_SIZE,
    hidden_size=HIDDEN_SIZE,
    output_size=OUTPUT_SIZE,
    max_tgt_len=MAX_TGT_LEN,
    dropout_p=0.0,
    bi_encoder=True,
    device=device
)

# 3. Train Model

In [7]:
trainer = Trainer(
    model=model,
    dataset=dataset,
    device=device,
    print_interval=1,
    plot_interval=-1,
    checkpoint_interval=10,
    expr_path=EXPR_PATH
)

In [8]:
trainer.train(num_epoch=10, batch_size=BATCH_SIZE)

Start to train
epoch:  1 ( 10%) time:    0m 21s (-      3m 9s) loss:2219.0129
epoch:  2 ( 20%) time:    0m 42s (-     2m 51s) loss:1658.9459
epoch:  3 ( 30%) time:     1m 3s (-     2m 28s) loss:1488.8841
epoch:  4 ( 40%) time:    1m 23s (-      2m 4s) loss:1375.5662
epoch:  5 ( 50%) time:    1m 44s (-     1m 44s) loss:1285.6540
epoch:  6 ( 60%) time:     2m 4s (-     1m 23s) loss:1208.4589
epoch:  7 ( 70%) time:    2m 25s (-      1m 2s) loss:1139.2452
epoch:  8 ( 80%) time:    2m 45s (-     0m 41s) loss:1075.5966
epoch:  9 ( 90%) time:     3m 7s (-     0m 20s) loss:1016.1490
epoch: 10 (100%) time:    3m 27s (-      0m 0s) loss:960.6209


# 4. Evaluate Model

In [9]:
evaluator = Evaluator(dataset, model, device=device)
evaluator.loadModel(EXPR_PATH+'ep10.model')

In [10]:
pairs, attn_list = evaluator.evalModel(num=10, beam_size=5)
for p, attn in zip(pairs, attn_list):
    print('Input : ' + ' '.join(p[0]))
    print('Gen   : ' + ' '.join(p[1]))
    print()

Input : 국제 전화 요 ? <EOS>
Gen   : 물론 이 ㅂ니다 . <EOS>

Input : 오전 7 시 부터 이 ㅂ니다 . <EOS>
Gen   : 알 겠 습니다 . <EOS>

Input : 따로 예약 을 안 하 시 었 나요 ? <EOS>
Gen   : 아니 요 . <EOS>

Input : 15 $ 입 니 다 . <EOS>
Gen   : 알 겠 습니다 . <EOS>

Input : 알 겠 습니다 . <EOS>
Gen   : 즐겁 ㄴ 여행 되 시 어요 . <EOS>

Input : 잘 되 었 네요 . 감사 하 ㅂ니다 . <EOS>
Gen   : 즐겁 ㄴ 여행 되 시 어요 . <EOS>

Input : 그렇 군요 , 감사 하 ㅂ니다 . <EOS>
Gen   : 천 만에 요 . <EOS>

Input : 무엇 을 돕 아 드리 ㄹ까요 ? <EOS>
Gen   : 음식 이 잘못 나오 았 나요 ? <EOS>

Input : 유튜브 동영상 보 시 었 어요 ? <EOS>
Gen   : 네 , 맞 습니다 . <EOS>

Input : 네 . 덕분 에 . <EOS>
Gen   : 감사 하 ㅂ니다 . <EOS>

